<a href="https://colab.research.google.com/github/Nav-Pallav/AV-Jobathon-march-2022-All-classification-tree-models-used-/blob/main/Missed_call_tracker_v0_4_(deduped_calls_returned).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

df=pd.read_csv("/content/drive/MyDrive/Hackathons/Text_summarization/Exotel Logs Inbound & Outboud- 15 Jun.csv")
#Test Data
#Exotel Logs Inbound & Outboud- 15 Jun
df=df[['CREATED_DATE','CALLER_TYPE','CALL_TYPE','CONNECTED_WITH','FROM_NUMBER']]
df.dropna(inplace=True)
#df=df.dropna(subset=['CREATED_DATE','APPOINTMENT'])
df['CREATED_DATE']= pd.to_datetime(df['CREATED_DATE'])
#df['APPOINTMENT']=df['APPOINTMENT'].astype(int)
df=df[df['CALLER_TYPE'].isin(['Connectcustomer', 'ConnectRM'])]

df['Cust_no'] = df.apply(lambda x: x['FROM_NUMBER'] if x['CALLER_TYPE'] == 'Connectcustomer' else x['CONNECTED_WITH'], axis=1).astype(int)
df['Rep_no'] = df.apply(lambda x: x['CONNECTED_WITH'] if x['CALLER_TYPE'] == 'Connectcustomer' else x['FROM_NUMBER'], axis=1).astype(int)
#df['APPOINTMENT']=df['Cust_no'].astype(int)

reps=df.Rep_no.unique()
rep_list=pd.DataFrame(reps, columns = ['Rep_no'])

In [ ]:
incomp=df[(df['CALLER_TYPE']=='Connectcustomer') & (df['CALL_TYPE']=='incomplete')].copy()
callbacks = df[(df['CALL_TYPE']=='completed')].copy()
rm_failed=df[(df['CALLER_TYPE']=='ConnectRM') & (df['CALL_TYPE']=='incomplete')].copy()
callbacks=pd.concat([callbacks, rm_failed])


incomp = incomp.sort_values(['CREATED_DATE','Cust_no','Rep_no'],
              ascending = [True, True,True])
callbacks = callbacks.sort_values(['CREATED_DATE','Cust_no','Rep_no'],
              ascending = [True, True,True])
callbacks['call_ret_time']=callbacks['CREATED_DATE']
callbacks['call_returned_flag']=1

In [ ]:
tol = timedelta(minutes=120)
missedcalls = pd.merge_asof(incomp,callbacks[['CREATED_DATE','Cust_no','Rep_no','call_returned_flag','call_ret_time']],on='CREATED_DATE',by=['Cust_no','Rep_no'],direction='forward',tolerance=tol)
missedcalls.fillna(0,inplace=True)
#out=missedcalls.groupby('Rep_no')['Cust_no'].count().reset_index().rename(columns={'Cust_no':'missed_processed'})

In [ ]:
missed_raw=incomp.groupby('Rep_no')['Cust_no'].count().reset_index().rename(columns={'Cust_no':'missed_raw'})
rec_raw_df=df[(df['CALLER_TYPE']=='Connectcustomer')].copy()
rec_raw=rec_raw_df.groupby('Rep_no')['Cust_no'].count().reset_index().rename(columns={'Cust_no':'received_raw'})

##callbacks_completed = df[(df['CALL_TYPE']=='completed')].copy()
##callback_processed=callbacks_completed.groupby('CONNECTED_WITH')['APPOINTMENT'].count().reset_index().rename(columns={'APPOINTMENT':'callback_processed'})

In [ ]:
######Missed#########

tin=missedcalls[missedcalls['call_returned_flag']==0]

tin=tin.sort_values(['CREATED_DATE','Cust_no','Rep_no'],
              ascending = [True, True,True])
tin['key']=tin['Cust_no'].astype(str) + tin['Rep_no'].astype('str')
tin['count_flag']=0
tin['window_end']=np.nan
key_list=list(tin['key'].drop_duplicates())
key_list
tol = timedelta(minutes=120)

tin_mod=pd.DataFrame()
for key in key_list:
  tin_fil=tin[tin['key']==key].reset_index()
  # for index,row in tin_fil.iterrows():
  #   print(row['Cust_no'])
  tin_fil.loc[0,'window_end']=tin_fil.loc[0,'CREATED_DATE']+timedelta(minutes=120)
  tin_fil.loc[0,'count_flag']=1
  for i in range(1, len(tin_fil)):
    if (tin_fil.loc[i-1, 'window_end'] > tin_fil.loc[i, 'CREATED_DATE']):
      tin_fil.loc[i, 'window_end'] = tin_fil.loc[i-1, 'window_end']

    else:
      tin_fil.loc[i, 'window_end'] = tin_fil.loc[i, 'CREATED_DATE']+timedelta(minutes=120)
      tin_fil.loc[i, 'count_flag']=1
  tin_mod=tin_mod.append(tin_fil,ignore_index=True)
  #tin_mod = tin_mod.append(tin_fil)
#tin_mod=pd.concat(tin_mod)

Missed_proc=tin_mod.groupby('Rep_no')['count_flag'].sum().reset_index().rename(columns={'count_flag':'missed_unret_processed'})

###### Returned ###########

ret=missedcalls[missedcalls['call_returned_flag']==1]
ret.drop_duplicates(subset=['call_ret_time','Rep_no','Cust_no'], keep='last', inplace=True)
Ret_proc=ret.groupby('Rep_no')['Cust_no'].count().reset_index().rename(columns={'Cust_no':'missed_ret_processed'})
# tin_dup=tin.copy()
# tin=pd.merge_asof()

# missedcalls = pd.merge_asof(incomp,callbacks[['CREATED_DATE','Cust_no','Rep_no','call_returned_flag']],on='CREATED_DATE',by=['Cust_no','Rep_no'],direction='forward',tolerance=tol)


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
reps=rep_list.merge(Missed_proc,on='Rep_no',how='left').fillna(0)
reps=reps.merge(missed_raw,on='Rep_no',how='left').fillna(0)
reps=reps.merge(rec_raw,on='Rep_no',how='left').fillna(0)
reps=reps.merge(Ret_proc,on='Rep_no',how='left').fillna(0)
reps['missed_received_processed']=reps['missed_unret_processed']+reps['missed_ret_processed']

In [ ]:
reps

,Rep_no,missed_unret_processed,missed_raw,received_raw,missed_ret_processed,missed_received_processed
0,18002585656,68.0,106.0,2027.0,32.0,100.0
1,19311506344,0.0,0.0,0.0,0.0,0.0
2,9515158946,1.0,4.0,19.0,3.0,4.0
3,8266994401,11.0,16.0,35.0,2.0,13.0
4,9978475151,2.0,3.0,15.0,1.0,3.0
...,...,...,...,...,...,...
2697,19315912872,0.0,0.0,0.0,0.0,0.0
2698,9315256831,0.0,0.0,1.0,0.0,0.0
2699,19926803969,0.0,0.0,0.0,0.0,0.0
2700,7011030984,0.0,0.0,1.0,0.0,0.0


In [ ]:
reps.to_csv('/content/drive/MyDrive/Hackathons/Text_summarization/output_1weekdata_v0.4.csv')